MediaGamma - Take Home Test
======

Qikai Gu

July 17, 2018


The test is about building a CTR prediction model with one of provided datasets. 
I picked [https://www.kaggle.com/c/avazu-ctr-prediction](https://www.kaggle.com/c/avazu-ctr-prediction).

It's expected to:
- finish the test in 2-3 hours
- return the results in 2 weeks but the sooner the better
- build an end-to-end pipeline for the task
- showcase my understanding of various aspects of ML: ETL, model building and selection, evaluation, etc.
- develop in python (jupyter notebooks) with reasonable comments
- use version control with appropriate commit messages

In [1]:
import numpy as np
import pandas as pd
import xlearn as xl
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Use only 'train.gz' here as we need labeled data to evaluate our models.
df = pd.read_csv('input/train.gz', compression='gzip', dtype={'id': str}, index_col='id')

In [3]:
pd.set_option('display.max_columns', 100)
df.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
id,,,,,,,,,,,,,,,,,,,,,,,
1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157
